# Análisis de sentimientos con NLP
Vamos a utilizar Spacy y scikit-learn para clasificar con conjunto de tweets en español como positivos/negativos

## Carga y preparación de los datos

In [1]:
import pandas as pd
import numpy as np

pd.set_option('display.max_colwidth', None) # leer máximo ancho de columna

# Leemos los datos
data_path ='./tweets_all.csv'
df = pd.read_csv(data_path, index_col=None)

df.head()

,content,polarity
0,"-Me caes muy bien \r\n-Tienes que jugar más partidas al lol con Russel y conmigo\r\n-Por qué tan Otako, deja de ser otako\r\n-Haber si me muero",NONE
1,"@myendlesshazza a. que puto mal escribo\r\n\r\nb. me sigo surrando help \r\n\r\n3. ha quedado raro el ""cómetelo"" ahí JAJAJAJA",N
2,@estherct209 jajajaja la tuya y la d mucha gente seguro!! Pero yo no puedo sin mi melena me muero,N
3,Quiero mogollón a @AlbaBenito99 pero sobretodo por lo rápido que contesta a los wasaps,P
4,Vale he visto la tia bebiendose su regla y me hs dado muchs grima,N


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1514 entries, 0 to 1513
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   content   1514 non-null   object
 1   polarity  1514 non-null   object
dtypes: object(2)
memory usage: 23.8+ KB


In [3]:
df.polarity.value_counts()

N       637
P       474
NEU     202
NONE    201
Name: polarity, dtype: int64

Tenemos 1514 tweets, de los cuales hay 637 positivos y 474 negativos. El resto son neutros o no tienen polaridad clasificada.
Vamos a entrenar sólo con los positivos y negativos para utilizar un clasificador binario

In [4]:
df = df[(df['polarity']=='P') | (df['polarity']=='N')]

In [5]:
df.polarity.value_counts()

N    637
P    474
Name: polarity, dtype: int64

In [12]:
!pip install -U --user spacy
!python -m spacy download es_core_news_md --user


  Using cached https://github.com/explosion/spacy-models/releases/download/es_core_news_md-3.3.0/es_core_news_md-3.3.0-py3-none-any.whl (42.3 MB)
  Using cached spacy-3.3.2-cp39-cp39-win_amd64.whl (11.7 MB)
  Using cached thinc-8.0.17-cp39-cp39-win_amd64.whl (1.0 MB)
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_md')


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


## Limpieza de texto
Hacemos un pequeño pre-procesado del texto antes de extraer las características:  
- Quitamos las menciones y las URL del texto porque no aportan valor para el análisis de sentimientos.
- Los hashtag sí que pueden aportar valor así que simplemente quitamos el #.
- Quitamos los signos de puntuación y palabras menores de 3 caracteres.
- Por último quitamos todos los símbolos de puntuación del texto (que forman parte de un token).
- Lematizamos el texto y lo guardamos en otra columna para comparar resultados del clasificador. 

In [6]:
# Debes instalar las librerías necesarias
import re, string, spacy
nlp=spacy.load('es_core_news_md')  # carga el modelo en español es_core_news_md de la librería de spacy para hacer NLP

In [7]:
# Lista de stop-words específicos de nuestro corpus (aproximación). Nota*: El corpus es el conjunto de textos que sirven como base para el análisis lingüístico
stop_words = ['el', 'la', 'lo', 'los', 'las', 'un', 'una', 'unos', 'unas', 'me', 'a', 'de', 'se', 'te']

pattern2 = re.compile('[{}]'.format(re.escape(string.punctuation))) #selecciona símbolos de puntuación

def clean_text(text):
    """Limpiamos las menciones y URL del texto. Luego convertimos todo en tokens,
    eliminamos los tokens que son signos de puntuación y convertimos en
    minúsculasn. Para terminar, volvemos a convertir en cadena de texto"""

    text = re.sub(r'@[\w_]+|https?://[\w_./]+', '', text) #elimina menciones y URL
    tokens = nlp(text)
    tokens = [tok.lower_ for tok in tokens if not tok.is_punct and not tok.is_space]
    filtered_tokens = [pattern2.sub('', token) for token in tokens if not (token in stop_words)] #obvia stop_words y después quita signos de puntuación
    filtered_text = ' '.join(filtered_tokens)
    
    return filtered_text

def lemmatize_text(text):
    """Convertimos el texto a tokens, extraemos el lexema de cada token
    y volvemos a convertir en cadena de texto"""
    tokens = nlp(text)
    lemmatized_tokens = [tok.lemma_ for tok in tokens]
    lemmatized_text = ' '.join(lemmatized_tokens)
    
    return lemmatized_text
 

Probamos el funcionamiento de estas funciones sobre un tweet de ejemplo:

In [8]:
print('Original:\n',df.content[10])
print('\nLimpiado:\n',clean_text(df.content[10]))
print('\nLematizado:\n',lemmatize_text(clean_text(df.content[10])))

Original:
 Mg y pongo un adjetivo super repelente a vuestro nombre 

Limpiado:
 mg y pongo adjetivo super repelente vuestro nombre

Lematizado:
 mg y poner adjetivo super repelente vuestro nombre


Aplicamos limpieza a todos los tweets del DataFrame y creamos columna nueva con los lemas

In [9]:
df.content=df.content.apply(clean_text)

In [10]:
#Quitamos tweets vacíos después de la limpieza
df=df[df.content!='']

In [11]:
df["lemas"]=df.content.apply(lemmatize_text)

In [12]:
#Contamos el nº de palabras por tweet
df['content_words'] = [len(t.split(' ')) for t in df.content]

In [13]:
print(df.describe())
print(df.head())

       content_words
count    1111.000000
mean       12.166517
std         4.764943
min         3.000000
25%         8.000000
50%        12.000000
75%        16.000000
max        26.000000
                                                                               content  \
1  a que puto mal escribo b sigo surrando help 3 ha quedado raro cómetelo ahí jajajaja   
2            jajajaja tuya y d mucha gente seguro pero yo no puedo sin mi melena muero   
3                        quiero mogollón pero sobretodo por rápido que contesta wasaps   
4                          vale he visto tia bebiendose su regla y hs dado muchs grima   
5                      ah mucho más por supuesto solo que incluyo habías entendido mal   

  polarity  \
1        N   
2        N   
3        P   
4        N   
5        P   

                                                                                     lemas  \
1  a que puto mal escribir b seguir surrar help 3 haber quedar raro cómetelo ahí jajajajar 

Vemos que de media cada tweet tiene unas 12 palabras, con un máximo de 26 palabras.  
### Clasificador
Vamos a usar la librería scikit-learn para aplicar un clasificador binario sobre la polaridad. Aplicamos dos modelos distintos para extraer las características del texto, Bag-of-Words (BoW) y Term Frequency times Inverse Document Frequency (TF-IDF).  

Primero dividimos en conjunto de entrenamiento y test.

In [14]:
from sklearn.model_selection import train_test_split

# Split data into training and test sets
# Asignamos un 70% a training y un 30% a test
X_train, X_test, y_train, y_test = train_test_split(df['content'], 
                                                    df['polarity'],
                                                    test_size=0.3,
                                                    random_state=0)

In [15]:
print('Primera entrada de train:\n', X_train.iloc[0])
print('Polaridad:', y_train.iloc[0])
print('\nX_train shape:', X_train.shape)
print('\nX_test shape:', X_test.shape)

Primera entrada de train:
 estoy preparando muchas cosas para que especial sea muy especial
Polaridad: P

X_train shape: (777,)

X_test shape: (334,)


## Modelo Bag of Words
El BoW se implementa con la función `CountVectorizer` de `scikit-learn`

El objetivo de la bolsa de palabras es transformar el texto en una representación numérica que se pueda utilizar en algoritmos de aprendizaje automático. El proceso implica los siguientes pasos:

Tokenización: El texto se divide en tokens o palabras individuales.

Limpieza: Se eliminan las palabras vacías (stopwords) y se realizan otras operaciones de limpieza del texto como la lematización.

Creación del vocabulario: Se crea un vocabulario de todas las palabras únicas que aparecen en el conjunto de documentos.

Conteo de frecuencia: Se cuenta el número de veces que cada palabra aparece en cada documento.

Creación de la matriz de características: Se crea una matriz que representa cada documento como un vector de características, donde cada característica corresponde a una palabra del vocabulario y su valor es el número de veces que esa palabra aparece en el documento.

In [16]:
from sklearn.feature_extraction.text import CountVectorizer

# aprendemos el modelo CountVectorizer sobre el conjunto de train
vect = CountVectorizer()
vect.fit(X_train)

CountVectorizer()

Vemos el número de términos distintos que tiene el diccionario:

In [17]:
len(vect.get_feature_names_out())

3277

Creamos la matriz BoW del conjunto de entrenamiento

In [25]:
# transformamos documentos de train en matriz de características
X_train_vectorized = vect.transform(X_train)

train=X_train_vectorized.toarray()
print(train.shape)
sum(train[:][0])  # hay diez elementos encontrados para la característica (feature) primera

(777, 3277)


10

### Entrenamiento del modelo
Vamos a probar un clasificador Logistic Regression de scikit-learn para entrenar nuestro modelo

In [26]:
from sklearn.linear_model import LogisticRegression

modelLR = LogisticRegression(solver='liblinear')
#Entrenamos el modelo con el conjunto de train
modelLR.fit(X_train_vectorized, y_train)

LogisticRegression(solver='liblinear')

### Verificación del modelo
Para ver el rendimiento del modelo usamos el conjunto de test. Primero transformamos el conjunto de test a su matriz BoW mediante el vectorizador aprendido en TRAIN y aplicamos el modelo entrenado:

In [27]:
# Predecimos sobre el conjunto de test
X_test_vectorized = vect.transform(X_test)
X_test_vectorized.shape

(334, 3277)

In [28]:
prediccion = modelLR.predict(X_test_vectorized)
print(prediccion)

['N' 'N' 'N' 'P' 'N' 'N' 'P' 'N' 'P' 'N' 'N' 'P' 'P' 'P' 'N' 'N' 'N' 'P'
 'P' 'P' 'N' 'P' 'P' 'P' 'N' 'N' 'N' 'P' 'P' 'N' 'P' 'P' 'P' 'P' 'N' 'N'
 'N' 'N' 'N' 'P' 'P' 'N' 'N' 'N' 'N' 'N' 'P' 'N' 'N' 'N' 'N' 'P' 'P' 'P'
 'P' 'P' 'N' 'P' 'N' 'N' 'P' 'N' 'P' 'P' 'N' 'N' 'P' 'N' 'P' 'N' 'P' 'P'
 'P' 'N' 'N' 'N' 'P' 'P' 'N' 'N' 'N' 'P' 'P' 'P' 'N' 'N' 'N' 'P' 'N' 'N'
 'N' 'N' 'N' 'N' 'N' 'P' 'N' 'P' 'N' 'P' 'N' 'N' 'P' 'N' 'N' 'N' 'P' 'N'
 'P' 'N' 'P' 'P' 'N' 'N' 'N' 'P' 'P' 'N' 'N' 'N' 'N' 'N' 'P' 'N' 'P' 'N'
 'N' 'N' 'P' 'N' 'P' 'N' 'P' 'N' 'N' 'P' 'N' 'P' 'N' 'P' 'N' 'N' 'N' 'N'
 'P' 'N' 'P' 'P' 'P' 'P' 'P' 'N' 'P' 'N' 'N' 'P' 'P' 'P' 'P' 'N' 'N' 'N'
 'P' 'N' 'N' 'P' 'N' 'N' 'N' 'N' 'P' 'P' 'N' 'P' 'N' 'P' 'N' 'N' 'P' 'P'
 'P' 'P' 'P' 'N' 'N' 'P' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'P'
 'P' 'P' 'N' 'N' 'N' 'P' 'N' 'N' 'N' 'P' 'P' 'P' 'N' 'N' 'N' 'N' 'N' 'N'
 'N' 'P' 'P' 'N' 'N' 'P' 'N' 'N' 'N' 'N' 'N' 'P' 'N' 'P' 'N' 'N' 'P' 'N'
 'N' 'P' 'N' 'N' 'P' 'N' 'N' 'N' 'N' 'P' 'P' 'P' 'N

Vemos el resultado de la predicción y calculamos su precisión con distintas métricas.  
Ejemplo de predicción de algunas muestras:

In [29]:
pd.DataFrame({'texto':X_test, 'polaridad':y_test, 'predicción':prediccion}).sample(10)

,texto,polaridad,predicción
134,totalmente pobre gappy que inocente es no sabe con quien ha encontrado,N,N
1246,ha explotado vaso en mano que forma tan bonita empezar día,N,P
1033,que kinox no quiere zi yo zoy buena perzona,N,N
1191,isco con juande no seria titular y sabeis al mister le van mas tissones,N,N
799,he capturado mi primer gimnasio en pokemon go cierto es que es probable que no tarden ni 10 minutos en quitarmelo pero mola,P,N
1373,mi padre le ha dado manotazo mi móvil y gracias dios que que ha roto ha sido cristal templado,N,N
481,ojalá justin subiendo foto comiéndose boca con sofía para que os den por culo todas,N,N
645,tiene que ser entretenido tu seras primera en verlo amore mio 😘,P,P
1261,yo puedo cambiar opinión cara es más difícil porque vale pasta,N,N
941,y que sea yo es triste,N,N


Precisión del modelo (# predicciones correctas / Total de muestras)

In [30]:
from sklearn.metrics import accuracy_score

print('Accuracy (exactitud): ', accuracy_score(y_test, prediccion))

Accuracy (exactitud):  0.7215568862275449


Matriz de confusión (predicción -columnas- frente a etiquetas reales -filas-)

In [31]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test, prediccion)
pd.DataFrame(cm, index=('N_true','P_true'), columns=('N_pred','P_pred'))
#filas: True Label, columnas: Prediction

,N_pred,P_pred
N_true,154,38
P_true,55,87


### Veamos qué palabras son las más relevantes en el modelo

In [32]:
# obtenemos los nombres de las características numpy array
feature_names = np.array(vect.get_feature_names_out())

# Ordenamos los coeficientes del modelo
sorted_coef_index = modelLR.coef_[0].argsort()

# Listamos los 10 coeficientes menores y mayores
print('Menores Coefs:\n{}\n'.format(feature_names[sorted_coef_index[:10]]))
print('Mayores Coefs: \n{}'.format(feature_names[sorted_coef_index[:-11:-1]]))

Menores Coefs:
['ni' 'no' 'puto' 'triste' 'alguien' 'puta' 'sad' 'porque' 'algún' 'eso']

Mayores Coefs: 
['gran' 'gracias' 'genial' 'buena' 'bien' 'hacerlo' 'puedes' 'guapa'
 'muchas' 'tranquila']


## Otros modelos
Probamos con los modelos Naïve Bayes y un SGD lineal para ver si mejora

Los modelos MultinomialNB (Multinomial Naive Bayes) son una técnica de aprendizaje automático que se utiliza principalmente en problemas de clasificación de texto.
El modelo SGDClassifier es un modelo de clasificación lineal que utiliza el método de descenso de gradiente estocástico como método de optimización para ajustar los pesos del modelo en el entrenamiento, por defecto aplica SVM (support vector machines).

In [33]:
def train_predict_evaluate_model(classifier, 
                                 train_features, train_labels, 
                                 test_features, test_labels):
    '''Función que entrena y valida un clasificador sobre
    un conjunto especificado de entrenamiento y test.
    Devuelve la predicción sobre el conjunto de test'''
    # entrena modelo    
    classifier.fit(train_features, train_labels)
    # predice en test usando el modelo
    predictions = classifier.predict(test_features) 
    # evalúa el rendimiento del modelo   
    print('Accuracy (exactitud): ', accuracy_score(test_labels, predictions))
    return predictions 

In [34]:

from sklearn.naive_bayes import MultinomialNB  # Multinomial naive bayes
from sklearn.linear_model import SGDClassifier

# creamos los modelos
modelNB = MultinomialNB()
modelSVM = SGDClassifier(loss='hinge', max_iter=10000, tol=1e-5) # loss/cost function, tol es el error a partir del cual deja de entrenar

# entrenamos y evaluamos
X_test_vectorized=vect.transform(X_test)
print("Modelo Multinomial Naïve Bayes:")
prediccionNB = train_predict_evaluate_model(modelNB, 
                                 X_train_vectorized, y_train, 
                                 X_test_vectorized, y_test)
print("Modelo SVM lineal:")
prediccionSVM = train_predict_evaluate_model(modelSVM, 
                                 X_train_vectorized, y_train, 
                                 X_test_vectorized, y_test)

Modelo Multinomial Naïve Bayes:
Accuracy (exactitud):  0.7514970059880239
Modelo SVM lineal:
Accuracy (exactitud):  0.6706586826347305


### Ejercicio 1:
Vuelve a entrenar el modelo Logistic Regression usando la función `train_predict_evaluate_model` para comparar fácilmente con los resultados anteriores. ¿cuál funciona mejor de los tres?

In [35]:
### SOLUCIÓN
# Crea el modelo de regresión logística
modelLR = LogisticRegression(solver='liblinear')

# Entrena y evalúa el modelo utilizando la función train_predict_evaluate_model
train_predict_evaluate_model(modelLR, X_train_vectorized, y_train, X_test_vectorized, y_test)

Accuracy (exactitud):  0.7215568862275449


array(['N', 'N', 'N', 'P', 'N', 'N', 'P', 'N', 'P', 'N', 'N', 'P', 'P',
       'P', 'N', 'N', 'N', 'P', 'P', 'P', 'N', 'P', 'P', 'P', 'N', 'N',
       'N', 'P', 'P', 'N', 'P', 'P', 'P', 'P', 'N', 'N', 'N', 'N', 'N',
       'P', 'P', 'N', 'N', 'N', 'N', 'N', 'P', 'N', 'N', 'N', 'N', 'P',
       'P', 'P', 'P', 'P', 'N', 'P', 'N', 'N', 'P', 'N', 'P', 'P', 'N',
       'N', 'P', 'N', 'P', 'N', 'P', 'P', 'P', 'N', 'N', 'N', 'P', 'P',
       'N', 'N', 'N', 'P', 'P', 'P', 'N', 'N', 'N', 'P', 'N', 'N', 'N',
       'N', 'N', 'N', 'N', 'P', 'N', 'P', 'N', 'P', 'N', 'N', 'P', 'N',
       'N', 'N', 'P', 'N', 'P', 'N', 'P', 'P', 'N', 'N', 'N', 'P', 'P',
       'N', 'N', 'N', 'N', 'N', 'P', 'N', 'P', 'N', 'N', 'N', 'P', 'N',
       'P', 'N', 'P', 'N', 'N', 'P', 'N', 'P', 'N', 'P', 'N', 'N', 'N',
       'N', 'P', 'N', 'P', 'P', 'P', 'P', 'P', 'N', 'P', 'N', 'N', 'P',
       'P', 'P', 'P', 'N', 'N', 'N', 'P', 'N', 'N', 'P', 'N', 'N', 'N',
       'N', 'P', 'P', 'N', 'P', 'N', 'P', 'N', 'N', 'P', 'P', 'P

El Multinomial Naïve Bayes es el que ha dado mejores resultados de los tres, pero aún así son todos muy parecidos.

## Modelo TF-IDF

El modelo TF-IDF asigna un peso a cada palabra en un documento en función de su frecuencia en el documento y en todo el corpus. La idea detrás del modelo es que las palabras que aparecen con frecuencia en un documento pero raramente en el corpus en general tienen un mayor peso y, por lo tanto, son más importantes para ese documento en particular. Las palabras que aparecen con frecuencia en el corpus en general pero raramente en el documento, tienen un peso menor y, por lo tanto, se consideran menos importantes para ese documento en particular.

EL modelo TF-IDF se implementa con la función `TfidfVectorizer` de `scikit-learn`. Definimos una función para calcular el modelo TF-IDF y extraer las características del conjunto de entrenamiento.

In [36]:
from sklearn.feature_extraction.text import TfidfVectorizer

#definimos una función para ajustar el modelo y extraer las características (palabras de la matriz)
def tfidf_extractor(corpus):
    '''Función que genera un modelo TF-IDF sobre un corpus de texto
    El corpus debe ser una lista de textos (palabras separadas por espacios)
    Devuelve el modelo TF-IDF generado y el vector TF-IDF del corpus'''
    
    vectorizer = TfidfVectorizer()
    features = vectorizer.fit_transform(corpus)
    return vectorizer, features

Calculamos el modelo TF-IDF sobre el corpus de entrenamiento y con este modelo extraemos las matrices del conjunto de entrenamiento y de test.

In [37]:
#Creamos los vectores de características TF-IDF
tfidf_vectorizer, tfidf_train_features = tfidf_extractor(X_train)  
tfidf_test_features = tfidf_vectorizer.transform(X_test)

In [38]:
#Entrenamos los 3 clasificadores con las características TF-IDF
modelos = [('Logistic Regression', modelLR),
           ('Naive Bayes', modelNB),
           ('Linear SVM', modelSVM)]
for m, clf in modelos:
    print('Modelo {} con características TF-IDF'.format(m))
    tfidf_predictions = train_predict_evaluate_model(classifier=clf,
                                           train_features=tfidf_train_features,
                                           train_labels=y_train,
                                           test_features=tfidf_test_features,
                                           test_labels=y_test)

Modelo Logistic Regression con características TF-IDF
Accuracy (exactitud):  0.6946107784431138
Modelo Naive Bayes con características TF-IDF
Accuracy (exactitud):  0.6916167664670658
Modelo Linear SVM con características TF-IDF
Accuracy (exactitud):  0.718562874251497


Obtenemos unos resultados algo peores a los que obtenemos con los modelos BoW.  
Los mayores coeficientes para cada clase son:

In [39]:
# obtenemos los nombres de las características numpy array
feature_names = np.array(tfidf_vectorizer.get_feature_names_out())

# Ordenamos los coeficientes del modelo
sorted_coef_index = modelLR.coef_[0].argsort()  

# Listamos los 10 coeficientes menores y mayores
print('Menores Coefs:\n{}\n'.format(feature_names[sorted_coef_index[:10]]))

print('Mayores Coefs: \n{}'.format(feature_names[sorted_coef_index[:-11:-1]])) # 10 últimos elementos, -1 significa en orden inverso


Menores Coefs:
['no' 'porque' 'ni' 'estoy' 'eso' 'triste' 'he' 'sad' 'puta' 'puto']

Mayores Coefs: 
['gracias' 'gran' 'feliz' 'día' 'buena' 'genial' 'bien' 'buen' 'mejor'
 'guapa']


In [40]:
# Convierto tfidf_train_features a array para visualizar su contenido y Comprueba las dimensiones de las los conjuntos de características de entrenamiento y test
train= tfidf_train_features.toarray()
print(tfidf_train_features.shape)
print(tfidf_test_features.shape)

(777, 3277)
(334, 3277)


### Ejercicio 2:
Crea una función `bow_extractor` análoga a la función `tfidf_extractor` para generar un modelo BoW a partir de un corpus de texto

In [41]:
#definimos una función para ajustar el modelo y extraer las características (palabras de la matriz)
def bow_extractor(corpus):
    '''Función que genera un modelo BoW sobre un corpus de texto
    El corpus debe ser una lista de textos (palabras separadas por espacios)
    Devuelve el modelo BoW generado y el vector BoW del corpus'''
    
    vectorizer = CountVectorizer()
    features = vectorizer.fit_transform(corpus)
    return vectorizer, features


## Modelos sobre texto lematizado
Probamos a entrenar los clasificadores con el texto lematizado para ver si mejoramos los resultados

In [42]:
X_train, X_test, y_train, y_test = train_test_split(df['lemas'], 
                                                    df['polarity'],
                                                    test_size=0.3,
                                                    random_state=0)

### Ejercicio 3:
Aplica los modelos de BoW y TF-IDF con las funciones definidas anteriormente para obtener las matrices de características del conjunto de entrenamiento y de test (texto lematizado). Tienes que calcular las matrices `bow_train_features` y `bow_test_features` con el modelo BoW y las matrices `tfidf_train_features` y `tfidf_test_features` sobre el modelo TFIDF

In [43]:
## SOLUCIÓN
#Modelo BoW
bow_vectorizer, bow_train_features = tfidf_extractor(X_train)  
bow_test_features = bow_vectorizer.transform(X_test)

#Modelo TF-IDF
tfidf_vectorizer, tfidf_train_features = tfidf_extractor(X_train)  
tfidf_test_features = tfidf_vectorizer.transform(X_test)

In [44]:
print(len(bow_vectorizer.get_feature_names_out()))
print(len(tfidf_vectorizer.get_feature_names_out()))

2618
2618


Observa que el número de términos en el vocabulario se ha reducido notablemente al coger el lema de las palabras (muchos términos compartían el mismo lema).  
Probamos si se mejora con los 3 clasificadores que hemos usado anteriormente.

In [45]:
#entrenamos clasificadores con modelos BoW
for m, clf in modelos:
    print('Modelo {} con características BoW'.format(m))
    bow_predictions = train_predict_evaluate_model(classifier=clf,
                                           train_features=bow_train_features,
                                           train_labels=y_train,
                                           test_features=bow_test_features,
                                           test_labels=y_test)

Modelo Logistic Regression con características BoW
Accuracy (exactitud):  0.7604790419161677
Modelo Naive Bayes con características BoW
Accuracy (exactitud):  0.7365269461077845
Modelo Linear SVM con características BoW
Accuracy (exactitud):  0.7125748502994012


Haz lo mismo para TF-IDF

In [46]:
# SOLUCIÓN
for m, clf in modelos:
    print('Modelo {} con características TF-IDF'.format(m))
    tfidf_predictions = train_predict_evaluate_model(classifier=clf,
                                           train_features=tfidf_train_features,
                                           train_labels=y_train,
                                           test_features=tfidf_test_features,
                                           test_labels=y_test)

Modelo Logistic Regression con características TF-IDF
Accuracy (exactitud):  0.7604790419161677
Modelo Naive Bayes con características TF-IDF
Accuracy (exactitud):  0.7365269461077845
Modelo Linear SVM con características TF-IDF
Accuracy (exactitud):  0.7305389221556886


Vemos que la clasificación sí que ha mejorado.  
Vemos cuáles son las características más importantes para el modelo LR sobre TF-IDF:

In [47]:
# obtenemos los nombres de las características numpy array
feature_names = np.array(tfidf_vectorizer.get_feature_names_out())

# Ordenamos los coeficientes del modelo
sorted_coef_index = modelLR.coef_[0].argsort()

# Listamos los 10 coeficientes menores y mayores
print('Menores Coefs:\n{}\n'.format(feature_names[sorted_coef_index[:10]]))
print('Mayores Coefs: \n{}'.format(feature_names[sorted_coef_index[:-11:-1]]))

Menores Coefs:
['no' 'porque' 'ni' 'triste' 'ese' 'poner' 'malo' 'sad' 'pobre' 'puto']

Mayores Coefs: 
['buen' 'gran' 'gracia' 'genial' 'feliz' 'tranquilo' 'mejor' 'mucho'
 'bien' 'guapo']


### EJERCICIO 4
Haz un análisis de subjetividad de textos, para ello trata de capturar texto de diferentes páginas web y compara el nivel de subjetividad de cada uno de ellos. Investiga qué librerías existen en python para realizar dicho análisis. ¿Qué índices proporcionan estas librerías y en qué se basan?

--------------------------------

Existen diversas librerías en Python que permiten realizar análisis de subjetividad de textos. Algunas de las más populares son:

* NLTK (Natural Language Toolkit): una librería de procesamiento de lenguaje natural que incluye herramientas para realizar análisis de sentimiento y subjetividad. Ofrece modelos pre-entrenados para distintos idiomas y dominios, así como funciones para entrenar modelos propios.

* TextBlob: una librería que ofrece una interfaz sencilla para realizar análisis de sentimiento y subjetividad. Incluye modelos pre-entrenados para inglés y permite entrenar modelos propios a partir de corpus etiquetados.

* VaderSentiment: una librería especializada en análisis de sentimiento para redes sociales y otros textos informales. Utiliza un conjunto de reglas heurísticas para identificar expresiones positivas, negativas y neutrales.

* Pattern: una librería que ofrece diversas funcionalidades para procesamiento de lenguaje natural, incluyendo análisis de sentimiento y subjetividad. Utiliza un modelo probabilístico basado en el análisis de frecuencia de palabras y expresiones.

Cada una de estas librerías proporciona distintos índices y medidas para el análisis de subjetividad. Algunos de los más comunes son:

* Polaridad: indica el grado de positividad o negatividad de un texto, en una escala que va de -1 a 1.

* Subjetividad: indica el grado de subjetividad o objetividad de un texto, en una escala que va de 0 a 1.

* Intensidad: indica el grado de intensidad emocional de un texto, en una escala que va de 0 a 1.

Estos índices se basan en modelos estadísticos y algoritmos de aprendizaje automático que han sido entrenados con _corpus_ etiquetados. Cada librería utiliza distintas estrategias y enfoques para realizar el análisis de subjetividad, por lo que es importante evaluar su desempeño en distintos tipos de textos y contextos.

In [48]:
import nltk
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     C:\Users\danie\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\abc.zip.
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     C:\Users\danie\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     C:\Users\danie\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping taggers\averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     C:\Users\danie\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers\averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     C:\Users\danie\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping grammars\basque_grammars.zip.
[nltk_data]   

True

Vamos a analizar una misma noticia presentada en diferentes periódicos. Para ello, hemos de tener en cuenta los siguientes listados:

_Periódicos de tendencia política de izquierda_:

* El País
* Público
* La Vanguardia (centro-izquierda)
* El Diario
* Infolibre
* El Salto

_Periódicos de tendencia política de derecha_:

* ABC
* El Mundo
* La Razón
* Libertad Digital
* Okdiario
* El Español

In [3]:
import requests
from bs4 import BeautifulSoup
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.sentiment import SentimentIntensityAnalyzer
import nltk


In [28]:
# Noticia de El País
url_1 = "https://elpais.com/sociedad/2023-04-05/la-fiscalia-archiva-la-investigacion-por-los-canticos-machistas-del-elias-ahuja.html "

# Noticia Libertad_digital
url_2 ="https://www.libertaddigital.com/madrid/2023-04-05/la-fiscalia-archiva-la-investigacion-por-los-canticos-del-colegio-mayor-elias-ahuja-al-no-constituir-delito-de-odio-7002206/"

try:
    page = requests.get(url_1)
except:
    print("Error al abrir la URL")

soup = BeautifulSoup(page.text, 'html.parser')

# Buscamos el <div> correspondiente y sacamos su contenido:
content_1 = soup.find('div', {"class": "a_c clearfix"})
article_1 = []
for i in content_1.find_all('p'):
    article_1.append(i.text)

article_1 = ''.join(article_1)

tokenizer = RegexpTokenizer(r'\w+')  # tokenizador para separar en palabras
stop_words = set(stopwords.words('spanish'))  # lista de stopwords
lemmatizer = WordNetLemmatizer()  # lematizador

# Separar el texto en palabras
words = tokenizer.tokenize(article_1.lower())

# Filtrar stopwords y lematizar las palabras
words = [lemmatizer.lemmatize(word, pos='v') for word in words if word not in stop_words]

# Analizar la subjetividad del texto
sia = SentimentIntensityAnalyzer()
polarity_scores = sia.polarity_scores(' '.join(words))

print(polarity_scores)


{'neg': 0.0, 'neu': 0.99, 'pos': 0.01, 'compound': 0.5859}


In [26]:
try:
    page = requests.get(url_2)
except:
    print("Error al abrir la URL")

soup = BeautifulSoup(page.text, 'html.parser')

# Buscamos el <div> correspondiente y sacamos su contenido:
content_2 = soup.find('div', {"class": "body"})
article_2 = []
for i in content_2.find_all('p'):
    article_2.append(i.text)

article_2 = ''.join(article_2)

tokenizer = RegexpTokenizer(r'\w+')  # tokenizador para separar en palabras
stop_words = set(stopwords.words('spanish'))  # lista de stopwords
lemmatizer = WordNetLemmatizer()  # lematizador

# Separar el texto en palabras
words = tokenizer.tokenize(article_2.lower())

# Filtrar stopwords y lematizar las palabras
words = [lemmatizer.lemmatize(word, pos='v') for word in words if word not in stop_words]

# Analizar la subjetividad del texto
sia = SentimentIntensityAnalyzer()
polarity_scores = sia.polarity_scores(' '.join(words))

print(polarity_scores)

{'neg': 0.0, 'neu': 0.992, 'pos': 0.008, 'compound': 0.2732}


In [45]:
import requests
from bs4 import BeautifulSoup
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.stem import SnowballStemmer

url = "https://elpais.com/sociedad/2023-04-05/la-fiscalia-archiva-la-investigacion-por-los-canticos-machistas-del-elias-ahuja.html"

try:
    page = requests.get(url)
except:
    print("Error al abrir la URL")

soup = BeautifulSoup(page.text, 'html.parser')


# Buscamos el <div> correspondiente y sacamos su contenido:
content = soup.find('div', {"class": "a_c clearfix"})
article = []
for i in content.find_all('p'):
    article.append(i.text)

article = ''.join(article)

tokenizer = RegexpTokenizer(r'\w+')  # tokenizador para separar en palabras
stop_words = set(stopwords.words('spanish'))  # lista de stopwords
stemmer = SnowballStemmer('spanish')  # lematizador para español

# Separar el texto en palabras
words = tokenizer.tokenize(article.lower())

# Filtrar stopwords y lematizar las palabras
words = [stemmer.stem(word) for word in words if word not in stop_words]

# Analizar la subjetividad del texto
sia = SentimentIntensityAnalyzer()
polarity_scores = sia.polarity_scores(' '.join(words))

print(polarity_scores)
print(words)

La Fiscalía archiva la investigación por los cánticos machistas del colegio mayor Elías Ahuja | Educación | EL PAÍSSeleccione:- - -EspañaAméricaMéxicoColombiaChileArgentinaUSAEducaciónsuscríbeteHHOLAIniciar sesiónEducaciónInfantil y PrimariaSecundaria, Bachillerato y FPUniversidadesÚltimas noticiasMACHISMOLa Fiscalía archiva la investigación por los cánticos machistas del colegio mayor Elías AhujaEl ministerio público considera que las expresiones fueron “irrespetuosas e insultantes para las mujeres”, pero no constituyen un delito de odio00:52"Putas, salid de vuestras madrigueras como conejas"Entrada al colegio mayor masculino Elías Ahuja, adscrito a la Universidad Complutense de Madrid.
Foto: CLAUDIO ÁLVAREZ | Vídeo: EPVElisa SilióLucía BohórquezMadrid / Palma - 05 abr 2023 - 09:37Actualizado: 05 abr 2023 - 10:02 UTCWhatsappFacebookTwitterCopiar enlaceComentariosLa Fiscalía de Madrid ha acordado archivar las diligencias de investigación abiertas contra un alumno de la residencia de es